# Recipe retrival!

I leanred how to web scrape responsibly from instructors Jonathan Kropko and Mojtaba Heidarysafa (UVA School of Data Science). As this code is being made publically availible, I modified it slightly so that it can not be download and run using my email in the header. Also, I am censoring a few links so that it is more difficult for this file to be used for evil!

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

## Getting the recipe links!
Scraping the website for the top recipes by ingredient over all ingredients. 

In [ ]:
headers = {'user-agent': 'MY_NAME UVA HooHacks Project (MY_EMAIL@virginia.edu)'} #EDITED FOR ANTI-REUSABILITY

In [164]:
#Base_url = 'https://tasty.co/ingredient' #I think I can get all of the recipes using recipe by ingredient
Base_url = None #ADD A BASE URL HERE! #EDITED FOR ANTI-REUSABILITY
r = requests.get(Base_url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')


In [165]:
test = soup.find_all(type="application/ld+json")


In [167]:
holder = test[0]
x=str(holder)[35:16902] #This is the part that is a json
import json
y = json.loads(x)
links_to_recipes_by_ing = [i['url'] for i in y["itemListElement"]]

In [317]:
links_to_recipes_by_ing[0:2] #Just peaking to see if it worked!

['https://tasty.co/ingredient/all-purpose-flour',
 'https://tasty.co/ingredient/allspice']

In [190]:
recipe_links = []
for link in links_to_recipes_by_ing:
    Base_url = link
    r = requests.get(Base_url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    test = soup.find_all(type="application/ld+json")
    try:
        holder = test[0]  
        holder
        x=str(holder)[35:-9]
        y = json.loads(x)
        recipe_links += [i['url'] for i in y["itemListElement"]]
    except:
        pass

Here I check for repeats by looking at the length of tthe list and the set

In [191]:
len(recipe_links)

3794

In [194]:
len(list(set(recipe_links)))

2098

Getting rid of the repeats

In [195]:
final_recipe_links = list(set(recipe_links))

## Scraping the instructions!

In [389]:
Base_url = final_recipe_links[44]
r = requests.get(Base_url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
test = soup.find_all(class_="xs-mb1")
grab_ingredients(test)
#print(test[2].get_text())

['Chicken Filling',
 '1 tablespoon neutral oil, such as canola or vegetable',
 '2 cloves garlic, minced',
 '¼ cup yellow onion (35 g), diced',
 '1 can fire-roasted diced green chiles, drained',
 '1 teaspoon ground cumin',
 '½ teaspoon dried oregano',
 '1 teaspoon kosher salt',
 '½ teaspoon freshly ground black pepper',
 '1 ½ cups shredded rotisserie chicken (190 g)',
 '¼ cup chicken stock (60 mL), or broth',
 'Beef Filling',
 '1 tablespoon neutral oil, such as canola or vegetable',
 '¼ cup yellow onion (35 g), diced',
 '1 clove garlic, minced',
 '2 teaspoons ground cumin',
 '1 teaspoon chili powder',
 '1 dried bay leaf',
 '½ teaspoon kosher salt, plus more to taste',
 '½ teaspoon freshly ground black pepper',
 '1 ½ cups shredded beef (190 g), or barbacoa',
 '½ cup beef broth (120 mL)',
 'Assembly',
 '16 corn tortillas',
 'nonstick cooking spray, for greasing',
 'For Serving',
 '¼ cup mexican crema (60 g)',
 '¼ cup crumbled queso fresco (35 g)',
 "Tasty's guacamole",
 "Tasty's pico de g

In [377]:
instructions = []
for link in final_recipe_links:
    Base_url = link
    r = requests.get(Base_url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    test = soup.find_all(class_="xs-mb2")
    maybe = test[2].get_text()
    if maybe.startswith('for'): #sometimes it gives "for __ serving(s)", so i check and take the next one
        instructions.append(test[3].get_text())
    else:
        instructions.append(test[2].get_text())

In [378]:
instructions

['PreparationFirst, make Chana Masala as follows:Heat avocado oil in a pan. To the pan, add bay leaves, dry chili pepper, cinnamon sticks and whole cumin seeds.Once cumin seeds start to sizzle, add diced onion and cook until slightly brown and translucent.Lower the heat and add the crushed or fresh tomatoes, ginger and garlic and mixwell.Add salt, turmeric, paprika, red chili powder, chana masala mix and water. Mix well.Let simmer for about 5 minutes with lid on.Add cooked chickpeas, mix well and let cook for 7-10 minutes.Then, add the taco seasoning to the chana masala you just made. Mash with masher and add water as needed for a loose but not runny consistency. Heat in a saucepan. Stir occasionally to prevent sticking.Serve mixture on taco shells with lettuce, tomatoes, avocado, optional cheese and salsa.Enjoy!',
 'PreparationPreheat oven to 160°C (325°F).In a medium saucepan, cook the apple, sugar, cinnamon, nutmeg, lemon zest, vanilla extract, and the juice of half a lemon until so

In [379]:
len(instructions)

2098

In [380]:
counter=0
for idx, instruct in enumerate(instructions):
    if instruct.startswith('Preparation'):
        instructions[idx] = instructions[idx][11:]

In [381]:
for idx, instruct in enumerate(instructions):
    if instruct.endswith('Enjoy!'):
        instructions[idx] = instructions[idx][:-6]

## Scraping the ingrdients!

In [315]:
def grab_ingredients(tags):
    """Input the list of tags from scraping
    Output the list of ingredients"""
    ls = []
    for tag in tags:
        ls.append(tag.get_text())
    ing = []
    for i in range(len(ls)):
        if ls[i].startswith('featured'):
            pass
        elif ls[i].startswith('Calories'):
            break
        else:
            ing.append(ls[i])
    return(ing)




ingredients = []
for link in final_recipe_links:
    Base_url = link
    r = requests.get(Base_url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    test = soup.find_all(class_="xs-mb1")
    ingredients.append(grab_ingredients(test))

In [323]:
len(ingredients)

2098

In [359]:
type(ingredients[0])

list

In [335]:
for lister in ingredients:
    #print(lister)
    for i in range(len(lister)):
        lister[i] = lister[i].replace(",","")

## Creating a dataframe and saving to a csv for later

In [382]:
data = pd.DataFrame({'recipe_link':final_recipe_links, 'ingredients':ingredients, 'instructions':instructions})

In [383]:
data.to_csv('scraped_recipe_data.csv')